# 딥드림 (DeepDream)

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

import matplotlib as mpl

import IPython.display as display
import PIL.Image

from tensorflow.keras.preprocessing import image

#### DeepDream을 적용할 이미지 선택

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'

In [ ]:
# 이미지를 다운로드해서 NumPy array로 읽기
def download(url, max_dim=None):
    name = url.split('/')[-1]
    image_path = tf.keras.utils.get_file(name, origin=url)
    img = PIL.Image.open(image_path)
    if max_dim:
      img.thumbnail((max_dim, max_dim))
    return np.array(img)

# [-1,1] 구간의 이미지를 [0,255] 이미지로 복구
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tf.cast(img, tf.uint8)

# 이미지를 화면에 출력
def show(img):
    display.display(PIL.Image.fromarray(np.array(img)))


# 이미지를 작업하기 편하게 크기를 줄임
original_img = download(url, max_dim=500)
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

## 특징을 추출할 베이스 모델 선택

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [ ]:
# 아래 지정된 계층의 출력을 최대화함
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

# 특징을 추출하기 위한 모델 생성
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

## Loss 계산

In [ ]:
def calc_loss(img, model):
    img_batch = tf.expand_dims(img, axis=0) # 이미지를 배치 형태로 변경 (배치 크기=1)
    layer_activations = model(img_batch)    # 모델 실행

    if len(layer_activations) == 1: # 선택한 계층이 하나인 경우 리스트로 만들어 줌
        layer_activations = [layer_activations]

    losses = []
    for act in layer_activations:
        loss = tf.math.reduce_mean(act)      # 각 계층 별 출력의 평균 계산
        losses.append(loss)

    return  tf.reduce_sum(losses)          # 계산된 loss를 합산

## DeepDream 정의

In [ ]:
class DeepDream(tf.Module):
    def __init__(self, model):
        self.model = model

    @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
    )
    def __call__(self, img, steps, step_size):
        print("Tracing")
        loss = tf.constant(0.0)
        for n in tf.range(steps):
            # 계층 별 Loss 계산
            with tf.GradientTape() as tape:
                # 이미지에 대해 Gradient Ascent해야 하므로 Gradient Tape이 이미지를 보도록 설정
                tape.watch(img) 
                loss = calc_loss(img, self.model)

            # 이미지에 대해 Gradient 계산
            gradients = tape.gradient(loss, img)

            # Gradient를 Normalize
            gradients /= tf.math.reduce_std(gradients) + 1e-8 

            # 이미지에 대해 Gradient Ascent
            # 지정된 계층의 출력이 커지면 Loss도 커지게 됨
            img = img + gradients*step_size
            img = tf.clip_by_value(img, -1, 1)  # pixel 값 clipping
        return loss, img

#### DeepDream 객체 생성

In [ ]:
deepdream = DeepDream(dream_model)

## DeepDream 실행

In [ ]:
def run_deep_dream_simple(img, steps=100, step_size=0.01):
    # uint8을 model의 입력 범위로 변환
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    img = tf.convert_to_tensor(img) # img를 Tensor로 변환
    step_size = tf.convert_to_tensor(step_size) # step_size를 Tensor로 변환
    
    steps_remaining = steps # 남은 실행 회수
    step = 0 # 실제 실행한 회수
    while steps_remaining:
        if steps_remaining>100:
            run_steps = tf.constant(100) # deepdeam 실행 회수 (최대 100씩 실행)
        else:
            run_steps = tf.constant(steps_remaining) # 100이하면 남은 회수 만큼 실행
        steps_remaining -= run_steps
        step += run_steps

        loss, img = deepdream(img, run_steps, tf.constant(step_size)) # DeepDream 실행

        display.clear_output(wait=True)
        show(deprocess(img))
        print ("Step {}, loss {}".format(step, loss))

    # 최종 결과 출력
    result = deprocess(img)
    display.clear_output(wait=True)
    show(result)

    return result

#### DeepDream 실행 함수 호출

In [ ]:
dream_img = run_deep_dream_simple(img=original_img, 
                                  steps=100, step_size=0.01)

## Multi-Scale Feature 생성

In [ ]:
import time
start = time.time()

OCTAVE_SCALE = 1.30

img = tf.constant(np.array(original_img))
base_shape = tf.shape(img)[:-1]
float_base_shape = tf.cast(base_shape, tf.float32)

for n in range(-2, 3):
    new_shape = tf.cast(float_base_shape*(OCTAVE_SCALE**n), tf.int32) # 옥타브 Scale의 새로운 크기 계산
    img = tf.image.resize(img, new_shape).numpy() # 이미지 resize
    img = run_deep_dream_simple(img=img, steps=50, step_size=0.01) # DeepDream 실행

display.clear_output(wait=True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)

end = time.time()
end-start

## Tiling 방식의 이미지 생성 (Advanced)
큰 이미지를 처리할 수 있도록 타일로 잘라서 처리하는 방식

#### 이미지 이동 : 좌우, 상하로 조금씩 이동시켜서 훈련 (Image Rolling)

In [ ]:
def random_roll(img, maxroll):
  # 타일 경계가 나타나지 않도록 이미지를 랜덤하게 이동
  shift = tf.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tf.int32)
  shift_down, shift_right = shift[0],shift[1] 
  img_rolled = tf.roll(tf.roll(img, shift_right, axis=1), shift_down, axis=0)
  return shift_down, shift_right, img_rolled

In [ ]:
shift_down, shift_right, img_rolled = random_roll(np.array(original_img), 512)
show(img_rolled)

#### 타일 방식의 그래디언트 계산

In [ ]:
class TiledGradients(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),)
  )
  def __call__(self, img, tile_size=512):
    shift_down, shift_right, img_rolled = random_roll(img, tile_size)

    # Initialize the image gradients to zero.
    gradients = tf.zeros_like(img_rolled)
    
    # 마지막 타일은 skip, (단, 타일이 1개인 경우는 제외) 
    xs = tf.range(0, img_rolled.shape[0], tile_size)[:-1] #  가로 방향의 타일 개수
    if not tf.cast(len(xs), bool): xs = tf.constant([0])
    ys = tf.range(0, img_rolled.shape[1], tile_size)[:-1] #  세로 방향의 타일 개수
    if not tf.cast(len(ys), bool): ys = tf.constant([0])

    # 타일 별로 그래디언트 계산
    for x in xs:
      for y in ys:
        with tf.GradientTape() as tape:
          tape.watch(img_rolled) # 이미지에 대해서 그래디언트 계산이 되도록 watch 처리

          #이미지에서 타일을 추출한 후 loss 계산
          img_tile = img_rolled[x:x+tile_size, y:y+tile_size]
          loss = calc_loss(img_tile, self.model)

        # 타일에 대해 Gradient Ascent
        gradients = gradients + tape.gradient(loss, img_rolled)

    # 그래디언트를 이미지 이동 전으로 위치로 복구
    gradients = tf.roll(tf.roll(gradients, -shift_right, axis=1), -shift_down, axis=0)

    # 그래디언트 정규화
    gradients /= tf.math.reduce_std(gradients) + 1e-8 

    return gradients 

In [ ]:
get_tiled_gradients = TiledGradients(dream_model)

#### 옥타브 스케일링 + 타일링 방식으로 이미지 생성

In [ ]:
def run_deep_dream_with_octaves(img, steps_per_octave=100, step_size=0.01, 
                                octaves=range(-2,3), octave_scale=1.3):
    base_shape = tf.shape(img)
    # img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img)

    initial_shape = img.shape[:-1]
    img = tf.image.resize(img, initial_shape)
    for octave in octaves:
        # 옥타브 Scale의 새로운 크기 계산
        new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
        img = tf.image.resize(img, tf.cast(new_size, tf.int32)) # 이미지 resize

        for step in range(steps_per_octave):
            gradients = get_tiled_gradients(img)  # 타일 방식의 그래디언트 계산
            img = img + gradients*step_size       # 이미지에 대해 Gradient Ascent
            img = tf.clip_by_value(img, -1, 1)    # pixel 값 clipping

            # 10번 마다 화면에 이미지 출력
            if step % 10 == 0:
                display.clear_output(wait=True)
                show(deprocess(img))
                print ("Octave {}, Step {}".format(octave, step))

    result = deprocess(img)
    return result

In [ ]:
img = run_deep_dream_with_octaves(img=original_img, step_size=0.01)

display.clear_output(wait=True)
base_shape = tf.shape(original_img)[:-1]
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)